In [28]:
import pandas as pd
import numpy as np

conflict_raw = pd.read_csv("conflict.csv")
GSEC1_19 = pd.read_csv("GSEC1_nl_dist.csv")
GSEC1_16 = pd.read_csv("GSEC1_nl_dist16.csv")
conflict_raw['event_date_date'] = pd.to_datetime(conflict_raw['event_date'])
conflict_raw['month'] = conflict_raw['event_date_date'].dt.month
conflict_raw['year'] = conflict_raw['event_date_date'].dt.year
conflict_raw['dist'] = conflict_raw['admin1'].str.lower()
conflict_raw['county'] = conflict_raw['admin2'].str.lower()

In [29]:
s1 = set(GSEC1_16['District'])
s2 = set(conflict_raw['dist'])
s1 - s2

{'amudat', 'kibaale', 'kibuku'}

In [30]:
distlist = list(set(conflict_raw['dist']))

In [31]:
event_types = list(set(conflict_raw['event_type']))
event_types

['Explosions/Remote violence',
 'Strategic developments',
 'Battles',
 'Violence against civilians',
 'Riots',
 'Protests']

In [32]:
conflict_raw.columns

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp', 'event_date_date', 'month', 'dist', 'county'],
      dtype='object')

In [33]:
counting_df = pd.DataFrame({"year":np.repeat([2019,2020],12),'month':list(range(1,13))*2,
                            'Strategic developments':0,'Battles':0,'Riots':0,'Explosions/Remote violence':0,
                 'Violence against civilians':0,  'Protests':0,'fatalities':0 })
counting_df['District'] = distlist[0]
num0dist = distlist[0]
for event in event_types:
    for ye in [2019,2020]:
        for mon in range(1,13):
            yea = counting_df['year'] == ye
            mont = counting_df['month'] == mon
            counting_df.loc[yea*mont,event] = conflict_raw.query("year == @ye and month == @mon and dist == @num0dist and event_type == @event").shape[0]
for ye in [2019,2020]:
    for mon in range(1,13):
        yea = counting_df['year'] == ye
        mont = counting_df['month'] == mon
        counting_df.loc[yea*mont,'fatalities'] = conflict_raw.query("year == @ye and month == @mon and dist == @num0dist ")['fatalities'].sum()
            
# count for the each type of conflict, and the total fatalities          
for i in distlist[1:]:
    res = pd.DataFrame({"year":np.repeat([2019,2020],12),'month':list(range(1,13))*2,
                            'Strategic developments':0,'Battles':0,'Riots':0,'Explosions/Remote violence':0,
                 'Violence against civilians':0,  'Protests':0,'fatalities':0 })
    res["District"] = i
    for event in event_types:
        for ye in [2019,2020]:
            for mon in range(1,13):
                yea = res['year'] == ye
                mont = res['month'] == mon
                res.loc[yea*mont,event] = conflict_raw.query("year == @ye and month == @mon and dist == @i and event_type == @event").shape[0]
                res.loc[yea*mont,'fatalities'] = conflict_raw.query("year == @ye and month == @mon and dist == @i")['fatalities'].sum()
    counting_df = pd.concat([counting_df,res],axis=0)


        
counting_df

,year,month,Strategic developments,Battles,Riots,Explosions/Remote violence,Violence against civilians,Protests,fatalities,District
0,2019,1,0,0,0,0,0,0,0,dokolo
1,2019,2,0,0,0,0,0,0,0,dokolo
2,2019,3,0,0,0,0,0,0,0,dokolo
3,2019,4,0,0,1,0,0,0,0,dokolo
4,2019,5,0,0,0,0,0,0,0,dokolo
...,...,...,...,...,...,...,...,...,...,...
19,2020,8,0,0,0,0,0,0,0,masaka
20,2020,9,0,0,0,0,0,0,0,masaka
21,2020,10,0,0,0,0,0,0,0,masaka
22,2020,11,0,0,2,0,1,0,0,masaka


In [34]:
len(set(counting_df['District']))

130

In [35]:
exportdf = pd.merge(GSEC1_19,counting_df,how="left",on=['District','year','month'])
for event in event_types:
    imput = exportdf[event].mode().values[0]
    exportdf[event] = exportdf[event].fillna(imput)
imput2 = exportdf['fatalities'].mode().values[0]
exportdf['fatalities'] = exportdf['fatalities'].fillna(imput2)
exportdf.to_csv("GSEC1_nl_conflict.csv",index=False)

In [36]:
testls = []
for event in event_types:
    val = exportdf.query("region == 4")[event].mode().values[0]
    testls.append(val)
testls.append(imput2)
testls

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

2016

In [37]:
counting_df = pd.DataFrame({"year":np.repeat([2016,2017],12),'month':list(range(1,13))*2,
                            'Strategic developments':0,'Battles':0,'Riots':0,'Explosions/Remote violence':0,
                 'Violence against civilians':0,  'Protests':0 ,'fatalities':0 })
counting_df['District'] = distlist[0]
num0dist = distlist[0]
for event in event_types:
    for ye in [2016,2017]:
        for mon in range(1,13):
            yea = counting_df['year'] == ye
            mont = counting_df['month'] == mon
            counting_df.loc[yea*mont,event] = conflict_raw.query("year == @ye and month == @mon and dist == @num0dist and event_type == @event").shape[0]
for ye in [2016,2017]:
    for mon in range(1,13):
        yea = counting_df['year'] == ye
        mont = counting_df['month'] == mon
        counting_df.loc[yea*mont,'fatalities'] = conflict_raw.query("year == @ye and month == @mon and dist == @num0dist ")['fatalities'].sum()
  
for i in distlist[1:]:
    res = pd.DataFrame({"year":np.repeat([2016,2017],12),'month':list(range(1,13))*2,
                            'Strategic developments':0,'Battles':0,'Riots':0,'Explosions/Remote violence':0,
                 'Violence against civilians':0,  'Protests':0,'fatalities':0  })
    res["District"] = i
    for event in event_types:
        for ye in [2016,2017]:
            for mon in range(1,13):
                yea = res['year'] == ye
                mont = res['month'] == mon
                res.loc[yea*mont,event] = conflict_raw.query("year == @ye and month == @mon and dist == @i and event_type == @event").shape[0]
                res.loc[yea*mont,'fatalities'] = conflict_raw.query("year == @ye and month == @mon and dist == @i")['fatalities'].sum()
    counting_df = pd.concat([counting_df,res],axis=0)
counting_df

,year,month,Strategic developments,Battles,Riots,Explosions/Remote violence,Violence against civilians,Protests,fatalities,District
0,2016,1,0,0,0,0,0,0,0,dokolo
1,2016,2,0,0,0,0,0,0,0,dokolo
2,2016,3,0,0,0,0,0,0,0,dokolo
3,2016,4,0,0,0,0,0,0,0,dokolo
4,2016,5,0,0,0,0,0,0,0,dokolo
...,...,...,...,...,...,...,...,...,...,...
19,2017,8,0,0,0,0,0,1,0,masaka
20,2017,9,0,0,1,0,0,1,0,masaka
21,2017,10,0,0,0,0,0,0,0,masaka
22,2017,11,0,0,0,0,0,0,0,masaka


In [38]:
exportdf = pd.merge(GSEC1_16,counting_df,how="left",on=['District','year','month'])
for event in event_types:
    imput = exportdf[event].mode().values[0]
    exportdf[event] = exportdf[event].fillna(imput)
imput2 = exportdf['fatalities'].mode().values[0]
exportdf['fatalities'] = exportdf['fatalities'].fillna(imput2)
                                             
exportdf.to_csv("GSEC1_nl_conflict16.csv",index=False)

In [40]:
testls = []
for event in event_types:
    val = exportdf.query("region == 4")[event].mode().values[0]
    testls.append(val)
testls.append(imput2)
testls

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]